# Span Batch Analysis - Data Preparation

## Initialization

Before start, Fill in [secret.yml](span-batch-analysis/secret.yaml). Refer to [secret_example.yml](span-batch-analysis/secret_example.yaml). This is for specifying RPC endpoints.

Also check config located at [config](span-batch-analysis/config/).

In [1]:
import os

Build binaries.

In [ ]:
%%bash
cd cmd/batch_decoder && go build . && cd ../../
cd cmd/span_batch_tester && go build . && cd ../../

In [2]:
%%capture
%env BATCH_DECODER=cmd/batch_decoder/batch_decoder
%env SPAN_BATCH_TESTER=cmd/span_batch_tester/span_batch_tester

Clone superchain config.

In [ ]:
%%bash
git clone https://github.com/ethereum-optimism/superchain-registry
cd superchain-registry && git checkout 5d3ca1966435827ac8c4216bddcb541ec64c1d6c && cd --

Select your chain.

In [3]:
%%capture
%env L1_CHAIN_NAME=mainnet
%env L2_CHAIN_NAME=base

# available networks
#      L1 - L2 list
# mainnet - base, op, pgn, zora
# goerli  - base, op, zora
# sepolia - op, pgn

In [4]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

start_number = !yq eval ".start_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
start_number_value = int(start_number[0])
end_number = !yq eval ".end_l1_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
end_number_value = int(end_number[0])
batch_tx_sender = !yq eval ".batch_tx_sender" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
batch_tx_sender_value = batch_tx_sender[0]
l2_genesis_timestamp = !yq eval ".genesis.l2_time" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_genesis_timestamp_value = int(l2_genesis_timestamp[0])
l2_chain_id = !yq eval ".chain_id" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
l2_chain_id_value = int(l2_chain_id[0])
inbox_addr = !yq eval ".batch_inbox_addr" "superchain-registry/superchain/configs/{l1_chain_name}/{l2_chain_name}.yaml"
inbox_addr_value = inbox_addr[0]

acc_start_number = !yq eval ".acc_l2_start_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_start_number_value = int(acc_start_number[0])
acc_end_number = !yq eval ".acc_l2_end_number" "span-batch-analysis/config/{l1_chain_name}/{l2_chain_name}.yaml"
acc_end_number_value = int(acc_end_number[0])

Read analyis config.

In [5]:
%env L1_START=$start_number_value
%env L1_END=$end_number_value
%env BATCH_TX_SENDER=$batch_tx_sender_value
%env L2_GENESIS_TIMESTAMP=$l2_genesis_timestamp_value
%env L2_CHAIN_ID=$l2_chain_id_value
%env INBOX=$inbox_addr_value

%env ACC_L2_START=$acc_start_number_value
%env ACC_L2_END=$acc_end_number_value


env: L1_START=18005000
env: L1_END=18025000
env: BATCH_TX_SENDER=0x5050F69a9786F081509234F1a7F4684b5E5b76C9
env: L2_GENESIS_TIMESTAMP=1686789347
env: L2_CHAIN_ID=8453
env: INBOX=0xff00000000000000000000000000000000008453
env: ACC_L2_START=3301000
env: ACC_L2_END=3302000


Get L1 RPC from secret.

In [6]:
key_name = f"L1_{l1_chain_name}_RPC".upper()
L1_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L1_RPC_ENDPOINT = L1_RPC[0]

key_name = f"L2_{l2_chain_name}_{l1_chain_name}_RPC".upper()
L2_RPC = !yq eval ".{key_name}" span-batch-analysis/secret.yaml
L2_RPC_ENDPOINT = L2_RPC[0]

In [7]:
%%capture
%env L1_RPC=$L1_RPC_ENDPOINT
%env L2_RPC=$L2_RPC_ENDPOINT

## Data Collection

All data will be stored at `./artifacts/{L1_CHAIN_NAME}/{L2_CHAIN_NAME}`

In [8]:
l1_chain_name = os.environ["L1_CHAIN_NAME"]
l2_chain_name = os.environ["L2_CHAIN_NAME"]

%env DB_PREFIX=./artifacts/$l1_chain_name/$l2_chain_name

env: DB_PREFIX=./artifacts/mainnet/base


batch decoder `fetch`: Read channel frames from L1 by reading batch transactions.

In [9]:
!$BATCH_DECODER fetch --l1=$L1_RPC --start=$L1_START --end=$L1_END --inbox=$INBOX --out=$DB_PREFIX/transactions_cache --sender=$BATCH_TX_SENDER --concurrent-requests=25 | tail -10

Fetched block:  18024962
Fetched block:  18024980
Fetched block:  18024983
Fetched block:  18024988
Fetched block:  18024964
Fetched block:  18024989
Fetched block:  18024951
Fetched batches in range [18005000,18025000). Found 3999 valid & 0 invalid batches
Fetch Config: Chain ID: 1. Inbox Address: 0xFf00000000000000000000000000000000008453. Valid Senders: map[0x5050F69a9786F081509234F1a7F4684b5E5b76C9:{}].
Wrote transactions with batches to ./artifacts/mainnet/base/transactions_cache


batch decoder `reassemble`: Reassemble channel frames to channels. 

In [10]:
!$BATCH_DECODER reassemble --inbox=$INBOX --in=$DB_PREFIX/transactions_cache --out=$DB_PREFIX/channel_cache | tail -10

span batch tester `convert`: Convert channels with singular batches to span batches. Try multiple tx encoding types.

In [11]:
!TX_ENCODE_TYPE=0  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[3990/3999] Channel ID: ff17565c00c4a65998c94425383adb2f, L2StartNum: 3265034, L2EndNum: 3265063, L2BlockCnt, 30
[3991/3999] Channel ID: ff37bd71dd602ea5fe5157c1f1d19025, L2StartNum: 3176811, L2EndNum: 3176840, L2BlockCnt, 30
[3992/3999] Channel ID: ff491053c9822ad16edefdf15643c5f6, L2StartNum: 3177140, L2EndNum: 3177169, L2BlockCnt, 30
[3993/3999] Channel ID: ff5095660abcbcb0224f5515e7d9b499, L2StartNum: 3218937, L2EndNum: 3218965, L2BlockCnt, 29
[3994/3999] Channel ID: ff896c648a4fcaab17a39575d14ce01c, L2StartNum: 3258326, L2EndNum: 3258355, L2BlockCnt, 30
[3995/3999] Channel ID: ff9493a3218f720b56703b97edfce912, L2StartNum: 3247178, L2EndNum: 3247207, L2BlockCnt, 30
[3996/3999] Channel ID: ffbc24e973a96562c3a8f3b785624694, L2StartNum: 3227306, L2EndNum: 3227335, L2BlockCnt, 30
[3997/3999] Channel ID: ffe79769e81d0fb85eb5a92f1aa14671, L2StartNum: 3239738, L2EndNum: 3239767, L2BlockCnt, 30
[3998/3999] Channel ID: fff1bf256e1bf054846d62167bf1bd23, L2StartNum: 3169334, L2EndNum: 3169363

In [12]:
!TX_ENCODE_TYPE=1  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[3990/3999] Channel ID: ff17565c00c4a65998c94425383adb2f, L2StartNum: 3265034, L2EndNum: 3265063, L2BlockCnt, 30
[3991/3999] Channel ID: ff37bd71dd602ea5fe5157c1f1d19025, L2StartNum: 3176811, L2EndNum: 3176840, L2BlockCnt, 30
[3992/3999] Channel ID: ff491053c9822ad16edefdf15643c5f6, L2StartNum: 3177140, L2EndNum: 3177169, L2BlockCnt, 30
[3993/3999] Channel ID: ff5095660abcbcb0224f5515e7d9b499, L2StartNum: 3218937, L2EndNum: 3218965, L2BlockCnt, 29
[3994/3999] Channel ID: ff896c648a4fcaab17a39575d14ce01c, L2StartNum: 3258326, L2EndNum: 3258355, L2BlockCnt, 30
[3995/3999] Channel ID: ff9493a3218f720b56703b97edfce912, L2StartNum: 3247178, L2EndNum: 3247207, L2BlockCnt, 30
[3996/3999] Channel ID: ffbc24e973a96562c3a8f3b785624694, L2StartNum: 3227306, L2EndNum: 3227335, L2BlockCnt, 30
[3997/3999] Channel ID: ffe79769e81d0fb85eb5a92f1aa14671, L2StartNum: 3239738, L2EndNum: 3239767, L2BlockCnt, 30
[3998/3999] Channel ID: fff1bf256e1bf054846d62167bf1bd23, L2StartNum: 3169334, L2EndNum: 3169363

In [13]:
!TX_ENCODE_TYPE=2  && $SPAN_BATCH_TESTER convert --in=$DB_PREFIX/channel_cache --out=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --l2=$L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[3990/3999] Channel ID: ff17565c00c4a65998c94425383adb2f, L2StartNum: 3265034, L2EndNum: 3265063, L2BlockCnt, 30
[3991/3999] Channel ID: ff37bd71dd602ea5fe5157c1f1d19025, L2StartNum: 3176811, L2EndNum: 3176840, L2BlockCnt, 30
[3992/3999] Channel ID: ff491053c9822ad16edefdf15643c5f6, L2StartNum: 3177140, L2EndNum: 3177169, L2BlockCnt, 30
[3993/3999] Channel ID: ff5095660abcbcb0224f5515e7d9b499, L2StartNum: 3218937, L2EndNum: 3218965, L2BlockCnt, 29
[3994/3999] Channel ID: ff896c648a4fcaab17a39575d14ce01c, L2StartNum: 3258326, L2EndNum: 3258355, L2BlockCnt, 30
[3995/3999] Channel ID: ff9493a3218f720b56703b97edfce912, L2StartNum: 3247178, L2EndNum: 3247207, L2BlockCnt, 30
[3996/3999] Channel ID: ffbc24e973a96562c3a8f3b785624694, L2StartNum: 3227306, L2EndNum: 3227335, L2BlockCnt, 30
[3997/3999] Channel ID: ffe79769e81d0fb85eb5a92f1aa14671, L2StartNum: 3239738, L2EndNum: 3239767, L2BlockCnt, 30
[3998/3999] Channel ID: fff1bf256e1bf054846d62167bf1bd23, L2StartNum: 3169334, L2EndNum: 3169363

span batch tester `analyze`: Analyze channels with singluar batches by comparing with corresponding span batches.

In [14]:
!TX_ENCODE_TYPE=0 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[3990/3999] Channel ID: ff17565c00c4a65998c94425383adb2f, CompressedReductionPercent: 6.490385 %
[3991/3999] Channel ID: ff37bd71dd602ea5fe5157c1f1d19025, CompressedReductionPercent: 6.579933 %
[3992/3999] Channel ID: ff491053c9822ad16edefdf15643c5f6, CompressedReductionPercent: 6.141326 %
[3993/3999] Channel ID: ff5095660abcbcb0224f5515e7d9b499, CompressedReductionPercent: 7.640014 %
[3994/3999] Channel ID: ff896c648a4fcaab17a39575d14ce01c, CompressedReductionPercent: 6.230066 %
[3995/3999] Channel ID: ff9493a3218f720b56703b97edfce912, CompressedReductionPercent: 6.391471 %
[3996/3999] Channel ID: ffbc24e973a96562c3a8f3b785624694, CompressedReductionPercent: 7.494848 %
[3997/3999] Channel ID: ffe79769e81d0fb85eb5a92f1aa14671, CompressedReductionPercent: 7.637616 %
[3998/3999] Channel ID: fff1bf256e1bf054846d62167bf1bd23, CompressedReductionPercent: 7.337095 %
[3999/3999] Channel ID: fff44887c7ce0dae0705b1fc986d56e8, CompressedReductionPercent: 6.365954 %


In [15]:
!TX_ENCODE_TYPE=1 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[3990/3999] Channel ID: ff17565c00c4a65998c94425383adb2f, CompressedReductionPercent: 3.596612 %
[3991/3999] Channel ID: ff37bd71dd602ea5fe5157c1f1d19025, CompressedReductionPercent: 3.353455 %
[3992/3999] Channel ID: ff491053c9822ad16edefdf15643c5f6, CompressedReductionPercent: 2.927491 %
[3993/3999] Channel ID: ff5095660abcbcb0224f5515e7d9b499, CompressedReductionPercent: 4.177178 %
[3994/3999] Channel ID: ff896c648a4fcaab17a39575d14ce01c, CompressedReductionPercent: 3.181560 %
[3995/3999] Channel ID: ff9493a3218f720b56703b97edfce912, CompressedReductionPercent: 3.914714 %
[3996/3999] Channel ID: ffbc24e973a96562c3a8f3b785624694, CompressedReductionPercent: 4.646127 %
[3997/3999] Channel ID: ffe79769e81d0fb85eb5a92f1aa14671, CompressedReductionPercent: 4.980092 %
[3998/3999] Channel ID: fff1bf256e1bf054846d62167bf1bd23, CompressedReductionPercent: 3.682335 %
[3999/3999] Channel ID: fff44887c7ce0dae0705b1fc986d56e8, CompressedReductionPercent: 3.690569 %


In [16]:
!TX_ENCODE_TYPE=2 && $SPAN_BATCH_TESTER analyze --in-channel=$DB_PREFIX/channel_cache --in-span-batch=$DB_PREFIX/span_batch_cache_v$TX_ENCODE_TYPE --out=$DB_PREFIX/result_v$TX_ENCODE_TYPE --chain-id=$L2_CHAIN_ID --tx-type=$TX_ENCODE_TYPE | tail -10

[3990/3999] Channel ID: ff17565c00c4a65998c94425383adb2f, CompressedReductionPercent: 8.379121 %
[3991/3999] Channel ID: ff37bd71dd602ea5fe5157c1f1d19025, CompressedReductionPercent: 8.184846 %
[3992/3999] Channel ID: ff491053c9822ad16edefdf15643c5f6, CompressedReductionPercent: 7.885764 %
[3993/3999] Channel ID: ff5095660abcbcb0224f5515e7d9b499, CompressedReductionPercent: 8.989033 %
[3994/3999] Channel ID: ff896c648a4fcaab17a39575d14ce01c, CompressedReductionPercent: 8.544818 %
[3995/3999] Channel ID: ff9493a3218f720b56703b97edfce912, CompressedReductionPercent: 8.045122 %
[3996/3999] Channel ID: ffbc24e973a96562c3a8f3b785624694, CompressedReductionPercent: 9.208776 %
[3997/3999] Channel ID: ffe79769e81d0fb85eb5a92f1aa14671, CompressedReductionPercent: 10.159262 %
[3998/3999] Channel ID: fff1bf256e1bf054846d62167bf1bd23, CompressedReductionPercent: 9.227049 %
[3999/3999] Channel ID: fff44887c7ce0dae0705b1fc986d56e8, CompressedReductionPercent: 8.192397 %


span batch tester `fetch`: Fetch consecutive singular batches.

In [22]:
!$SPAN_BATCH_TESTER fetch --l2 $L2_RPC --start=$ACC_L2_START --end=$ACC_L2_END --concurrent-requests=5 --out=$DB_PREFIX/singular_batches_cache | tail -10

Fetched L2 block:  3301992
Fetched L2 block:  3301993
Fetched L2 block:  3301994
Fetched L2 block:  3301995
Fetched L2 block:  3301996
Fetched L2 block:  3301998
Fetched L2 block:  3301997
Fetched L2 block:  3301999
Fetched v0 batches in range [3301000,3302000).
Fetch Config: Chain ID: 8453.


span batch tester `merge`: Merge singular batches:

This command will analyze `start - end` number of batches, Comparing uncompressed size of below two batches:
- For `i` in range  `0` to `start - end - 1`
  - singular batch from number `start` to `start + i`, inclusive.
  - span batch from number `start` to `start + i`, inclusive.

Upper command will make a result json file which has name of the form `{start_number}_{end_number}.json`.

In [18]:
!$SPAN_BATCH_TESTER merge --start=$ACC_L2_START --end=$ACC_L2_END --l2 $L2_RPC --genesis-timestamp=$L2_GENESIS_TIMESTAMP --in=$DB_PREFIX/singular_batches_cache --out=$DB_PREFIX/merge_result | tail -10

2023/08/30 21:30:38 open artifacts/mainnet/base/singular_batches_cache/3301018.json: no such file or directory
